# Database connection test

# Preparation

In [2]:
# Imports
import sqlite3
import pandas as pd

## Main connection

In [4]:
# Connect to main database
db_path = '../TsakonianDB.sqlite3'
conn = sqlite3.connect(db_path)

## Auxiliary functions

In [5]:
def query(query: str,
          cursor = conn.cursor()):

      cursor.execute(query)
      fetched = cursor.fetchall()

      if len(fetched) > 0:
            temp_df = pd.DataFrame(fetched, columns=[i[0] for i in cursor.description])
            return temp_df
      else:
            print('Query executed successfully. No results to show.')

# Add words from main dictionary to the database

## Load dictionary

In [6]:
# Load main dictionary
main_df_path = '../../data/tables/main.xlsx'
main_df = pd.read_excel(main_df_path)
main_df

,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άζ̌α,"αραία, όχι συχνά",NaN,1
3,άθρωπο,άνθροπος,Α1,1
4,άι,λάδι,Υ0,1
...,...,...,...,...
620,χορέ,χορός,Α,1
621,μοίρα,μοίρα,Θ,1
622,αέρα,αέρας,Α,1
623,λεξικό,λεξικό,Υ,1


## Manage duplicates

In [7]:
# Remove duplicates with the exact same information
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = main_df.drop_duplicates()
print(f'Entries after removing duplicates: {len(main_df)}')

Entries before removing duplicates: 625
Entries after removing duplicates: 625


In [8]:
# Show duplicates
main_df[main_df.duplicated(subset = 'tsakonian', keep = False)].sort_values(by = 'tsakonian')

,tsakonian,greek,paradigm,source_id
35,αβουτάνα,αυτί,Θ1,1
598,αβουτάνα,αυτί,NaN,1
138,βου,βόδι,Α2,1
624,βου,κλαίω,Ρ,1
144,βραχάνι,φουστάνι,Υ3,1
600,βραχάνι,μακρύ φουστάνι,NaN,1
260,κηρούνι,πηρούνι,Υ3,1
602,κηρούνι,πηρούνι,NaN,1
341,μπρούσα,τσέπι,Θ1,1
614,μπρούσα,τσέπη,Θ,1


In [9]:
# Keep the Tsakonian-Greek duplicate with the lowest ID
print(f'Entries before removing duplicates: {len(main_df)}')
main_df = (main_df
           .sort_index()
           .drop_duplicates(subset = ['tsakonian', 'greek'], keep = 'first')
           .sort_values(by = 'tsakonian')
           .reset_index(drop = True)
           )
print(f'Entries after removing duplicates: {len(main_df)}')
main_df

Entries before removing duplicates: 625
Entries after removing duplicates: 624


,tsakonian,greek,paradigm,source_id
0,άγο,άλογο,Α0,1
1,άγουστε,αύγουστος,NaN,1
2,άζ̌α,"αραία, όχι συχνά",NaN,1
3,άθρωπο,άνθροπος,Α1,1
4,άι,λάδι,Υ0,1
...,...,...,...,...
619,όντα,δόντι,Υ,1
620,όρεγι,εδώ ακριβώς,Ζ,1
621,όρπα,εκεί,Ζ,1
622,ότσι,ότι,Ζ,1


## Save to main Excel

In [10]:
# Save main_df back to the Excel file
main_df.to_excel(main_df_path, index = False)
print('Changes consolidated.')

Changes consolidated.


In [16]:
# Load main dictionary
main_df = pd.read_excel(main_df_path)

# Copy the full dictionary into the database
main_df.to_sql('dictionary_entry', 
               conn, 
               if_exists='replace', 
               index=False,
               dtype = {'tsakonian': 'varchar(50)',
                        'greek': 'varchar(200)',
                        'paradigm': 'varchar(5)',
                        'source_id': 'bigint',
               })

# Delete the temporary table if it exists
try:
    query("DROP TABLE sqlitestudio_temp_table;")
except:
    pass

# Add primary keys and foreign keys by recreating the table
# Obtained from SQLiteStudio
recreating_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                          FROM dictionary_entry;

DROP TABLE dictionary_entry;

CREATE TABLE dictionary_entry (
    id        INTEGER       PRIMARY KEY AUTOINCREMENT,
    tsakonian VARCHAR (50)  NOT NULL,
    greek     VARCHAR (200),
    paradigm  VARCHAR (5),
    source_id    INTEGER       REFERENCES dictionary_source (id) 
);

INSERT INTO dictionary_entry (
                                 tsakonian,
                                 greek,
                                 paradigm,
                                 source_id
                             )
                             SELECT tsakonian,
                                    greek,
                                    paradigm,
                                    source_id
                               FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()

Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.
Query executed successfully. No results to show.


In [ ]:
ing_query = """PRAGMA foreign_keys = 0;

CREATE TABLE sqlitestudio_temp_table AS SELECT *
                                          FROM dictionary_entry;

DROP TABLE dictionary_entry;

CREATE TABLE dictionary_entry (
    tsakonian VARCHAR (50)  PRIMARY KEY
                            NOT NULL,
    greek     VARCHAR (200),
    paradigm  VARCHAR (5),
    source_id    INTEGER       REFERENCES dictionary_source (id) 
);

INSERT INTO dictionary_entry (
                                 tsakonian,
                                 greek,
                                 paradigm,
                                 source_id
                             )
                             SELECT tsakonian,
                                    greek,
                                    paradigm,
                                    source_id
                               FROM sqlitestudio_temp_table;

DROP TABLE sqlitestudio_temp_table;

PRAGMA foreign_keys = 1;"""

# Execute queries in a loop
# Only one query can be executed at a time
for q in recreating_query.split(';'):
      query(q)

# Save changes
conn.commit()